In [1]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import sqlalchemy as sql
import re
from tqdm import tqdm

sys.path.append(r'..\2023-Internal Forecast QC Automation\Forecast QC Automation')
from data_prep_functions import *
from mgra_input_file_automation import *

In [2]:
years = ['2022', '2026', '2029', '2032', '2035', '2040', '2050']
version = '2023_11_10'

# Phase 1

## 0. Data Preparation

In [4]:
geo_dfs = {}
geo_dfs['mgra'] = mgra_denorm_path=mgra_output('SR15', False)
geo_dfs['mgra']['vacancy'] = geo_dfs['mgra']['hs'] - geo_dfs['mgra']['hh']
geo_dfs['mgra']['vacancy_rate'] = geo_dfs['mgra']['vacancy']/geo_dfs['mgra']['hs']
geo_dfs['mgra'] = geo_dfs['mgra'].set_index(['mgra', 'year'])
geo_dfs['mgra'].insert(0, 'geotype', 'mgra')

In [5]:
# compare columns. check if all columns are accounted for

for col in pd.read_csv(r'C:/Users/jchu/OneDrive - San Diego Association of Governments/Projects/2023/Series 15 Subregional Forecast/2023-083 MGRA Input File- Series 15 Subregional Baseline Forecast/Data/2022/2022_2023_11_10/mgra_based_input_2022_2023_11_10.csv').columns:
    if col not in list(geo_dfs['mgra'].columns):
        print(col)

year
mgra


### Geography roll-up with all forecast years (Region, Jurisdiction, CPA, LUZ)  

In [6]:
geo_levels = ['region', 'jurisdiction', 'cpa', 'luz', 'taz']
for geo in tqdm(geo_levels):
    geo_dfs[geo] = create_mgra_denorm_table(geo_dfs['mgra'].reset_index(), 
                         geo_level=geo,
                         output_folder_path=False, 
                         version='SR15',
                         series=15)
    if geo == 'cpa':
        geo_dfs[geo] = add_cpa_names(geo_dfs[geo], 15).set_index(['cpa', 'year'])
    geo_dfs[geo].insert(0, 'geotype', geo)
    display(geo_dfs[geo])
geo_dfs['jurisdiction']

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

geotype  scenario       taz      luz         zip  pseudomsa  \
region    year                                                               
San Diego 2022  region         0  61885698  2834354  2238859007     111419   
          2026  region         0  61885698  2834354  2238859007     111419   
          2029  region         0  61885698  2834354  2238859007     111419   
          2032  region         0  61885698  2834354  2238859007     111419   
          2035  region         0  61885698  2834354  2238859007     111419   
          2040  region         0  61885698  2834354  2238859007     111419   
          2050  region         0  61885698  2834354  2238859007     111419   

                district27      pop      hhp      gq  ...  redev_emp_res  \
region    year                                        ...                  
San Diego 2022      232154  3287306  3170895  116411  ...    5751.485158   
          2026      232154  3302237  3188816  113421  ...    3923.499753   
          2029      232154  3334675  3220394  114281  ...    3686.796230   
          2032      232154  3373033  3257421  115612  ...    3575.591472   
          2035      232154  3404362  3287304  117058  ...    3404.907336   
          2040      232154  3432211  3315776  116435  ...    3054.843543   
          2050      232154  3400250  3285676  114574  ...    2902.186610   

                redev_emp_emp    infill_sf   infill_mf   infill_emp  \
region    year                                                        
San Diego 2022    9816.537780  5077.680910  689.775228  8820.060976   
          2026    9374.358484  3919.240695  457.586141  8654.583028   
          2029    8979.347826  3729.214992  406.626178  8417.827502   
          2032    8790.898273  3639.102629  354.508863  8217.821170   
          2035    8128.687578  3312.415162  315.666382  7833.609627   
          2040    7623.962261  3242.902270  223.030615  7515.209493   
          2050    6827.913177  3002.764201  108.114246  6712.540908   

                         dev           vac       unusable  vacancy  \
region    year                                                       
San Diego 2022  2.087814e+06  63080.562020  114791.146477    75170   
          2026  2.100048e+06  51085.984387  114791.146477    85846   
          2029  2.102949e+06  48186.029811  114791.146477    92735   
          2032  2.105195e+06  45940.224893  114791.146477    94882   
          2035  2.107488e+06  43648.218842  114791.146477    96868   
          2040  2.109871e+06  41270.628084  114791.146477    99898   
          2050  2.112782e+06  38364.859834  114791.146477   103918   

                vacancy_rate  
region    year                
San Diego 2022      0.060835  
          2026      0.066673  
          2029      0.070253  
          2032      0.070441  
          2035      0.070558  
          2040      0.070819  
          2050      0.072242  

[7 rows x 126 columns]

 20%|████████████████▊                                                                   | 1/5 [00:03<00:15,  3.81s/it]

geotype  scenario     taz     luz       zip  \
jurisdiction year                                                     
Carlsbad     2022  jurisdiction         0  294881  126916  69743180   
             2026  jurisdiction         0  294881  126916  69743180   
             2029  jurisdiction         0  294881  126916  69743180   
             2032  jurisdiction         0  294881  126916  69743180   
             2035  jurisdiction         0  294881  126916  69743180   
...                         ...       ...     ...     ...       ...   
Vista        2029  jurisdiction         0  627759  135889  65655038   
             2032  jurisdiction         0  627759  135889  65655038   
             2035  jurisdiction         0  627759  135889  65655038   
             2040  jurisdiction         0  627759  135889  65655038   
             2050  jurisdiction         0  627759  135889  65655038   

                   pseudomsa  district27     pop     hhp    gq  ...  \
jurisdiction year                                               ...   
Carlsbad     2022       4550         859  115585  114619   966  ...   
             2026       4550         859  117853  116921   932  ...   
             2029       4550         859  121388  120450   938  ...   
             2032       4550         859  121713  120773   940  ...   
             2035       4550         859  123720  122767   953  ...   
...                      ...         ...     ...     ...   ...  ...   
Vista        2029       4928         713  101464   99458  2006  ...   
             2032       4928         713  102275  100263  2012  ...   
             2035       4928         713  104034  101976  2058  ...   
             2040       4928         713  105305  103261  2044  ...   
             2050       4928         713  104240  102356  1884  ...   

                   redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
jurisdiction year                                                       
Carlsbad     2022      48.981013     266.827288   2.975080  24.947334   
             2026      21.651900     240.779409   1.212635  12.087944   
             2029      15.077134     225.761543   0.000000  11.896798   
             2032       9.245455     212.204984   0.000000   0.000000   
             2035       1.491617     204.058286   0.000000   0.000000   
...                          ...            ...        ...        ...   
Vista        2029      90.628278     107.389934  10.431880   0.000000   
             2032      90.226848     106.556701   4.910551   0.000000   
             2035      89.908896     105.086247   0.000000   0.000000   
             2040      85.889377     103.759234   0.000000   0.000000   
             2050      85.889377      64.388351   0.000000   0.000000   

                   infill_emp           dev         vac     unusable  vacancy  \
jurisdiction year                                                               
Carlsbad     2022  627.031082  18724.597352  845.282086  5471.912863     3113   
             2026  595.960729  19054.612290  525.787043  5471.912863     3646   
             2029  536.986770  19158.010274  422.389059  5471.912863     3924   
             2032  479.737698  19170.574169  409.825165  5471.912863     3947   
             2035  425.903764  19220.800147  359.599186  5471.912863     3767   
...                       ...           ...         ...          ...      ...   
Vista        2029  487.146471  11040.145603  333.059170   539.573510     1651   
             2032  484.422395  11052.685378  320.519395   539.573510     1673   
             2035  480.802577  11069.661065  303.543707   539.573510     1716   
             2040  452.861250  11096.379180  276.825592   539.573510     2156   
             2050  368.918526  11167.652975  205.551797   539.573510     3350   

                   vacancy_rate  
jurisdiction year                
Carlsbad     2022      0.064714  
             2026      0.071671  
             2029      0.073854  
  

 40%|█████████████████████████████████▌                                                  | 2/5 [00:06<00:09,  3.22s/it]

...

geotype  scenario     taz       zip  pseudomsa  district27    pop  \
luz year                                                                     
1   2022     luz         0  221777  13631580        296        3996   7233   
    2026     luz         0  221777  13631580        296        3996   7162   
    2029     luz         0  221777  13631580        296        3996   7155   
    2032     luz         0  221777  13631580        296        3996   7155   
    2035     luz         0  221777  13631580        296        3996   7273   
...          ...       ...     ...       ...        ...         ...    ...   
245 2029     luz         0  169137   3588811        195         507  10565   
    2032     luz         0  169137   3588811        195         507   9694   
    2035     luz         0  169137   3588811        195         507   9676   
    2040     luz         0  169137   3588811        195         507   9849   
    2050     luz         0  169137   3588811        195         507  10289   

            hhp  gq  gq_mil  ...  redev_emp_res  redev_emp_emp  infill_sf  \
luz year                     ...                                            
1   2022   7191  42       0  ...       0.115001      17.192360   0.000000   
    2026   7121  41       0  ...       0.000000      17.192360   0.000000   
    2029   7114  41       0  ...       0.000000      17.192360   0.000000   
    2032   7114  41       0  ...       0.000000      17.192360   0.000000   
    2035   7231  42       0  ...       0.000000      17.192360   0.000000   
...         ...  ..     ...  ...            ...            ...        ...   
245 2029  10497  68       0  ...       3.840816      93.776818  93.047038   
    2032   9625  69       0  ...       3.840816      92.615707  90.385080   
    2035   9606  70       0  ...       0.369939      88.284980  80.604189   
    2040   9780  69       0  ...       0.369939      84.820773  76.973511   
    2050  10228  61       0  ...       0.000000      52.019728  59.161195   

          infill_mf  infill_emp         dev        vac   unusable  vacancy  \
luz year                                                                     
1   2022   0.000000    6.912115  876.825496  19.026480  59.487941      188   
    2026   0.000000    6.912115  882.809483  14.190585  59.487941      211   
    2029   0.000000    6.912115  882.809483  14.190585  59.487941      192   
    2032   0.000000    6.912115  882.809483  14.190585  59.487941      179   
    2035   0.000000    6.912115  882.809483  14.190585  59.487941      114   
...             ...         ...         ...        ...        ...      ...   
245 2029  22.499032    8.162603  689.161390  17.742480   0.000000      232   
    2032   6.587490    8.162603  691.953903  14.949968   0.000000      713   
    2035   0.000000    8.162603  694.461223  12.442648   0.000000      989   
    2040   0.000000    8.162603  695.031630  11.872241   0.000000      893   
    2050   0.000000    8.162603  698.477564   8.426307   0.000000      743   

          vacancy_rate  
luz year                
1   2022      0.049292  
    2026      0.054409  
    2029      0.049446  
    2032      0.045921  
    2035      0.029096  
...                ...  
245 2029      0.057654  
    2032      0.168717  
    2035      0.218419  
    2040      0.195619  
    2050      0.158456  

[1715 rows x 125 columns]

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:12<00:02,  3.00s/it]

geotype  scenario   luz     zip  pseudomsa  district27   pop   hhp  \
taz  year                                                                      
13   2022     taz         0   676  370071         24           4  4646  2859   
     2026     taz         0   676  370071         24           4  4570  2816   
     2029     taz         0   676  370071         24           4  4549  2788   
     2032     taz         0   676  370071         24           4  4535  2760   
     2035     taz         0   676  370071         24           4  4528  2741   
...           ...       ...   ...     ...        ...         ...   ...   ...   
4947 2029     taz         0  1808  735472         64         112     5     5   
     2032     taz         0  1808  735472         64         112     0     0   
     2035     taz         0  1808  735472         64         112     0     0   
     2040     taz         0  1808  735472         64         112     0     0   
     2050     taz         0  1808  735472         64         112     0     0   

             gq  gq_mil  ...  redev_emp_res  redev_emp_emp  infill_sf  \
taz  year                ...                                            
13   2022  1787    1787  ...            0.0            0.0        0.0   
     2026  1754    1754  ...            0.0            0.0        0.0   
     2029  1761    1761  ...            0.0            0.0        0.0   
     2032  1775    1775  ...            0.0            0.0        0.0   
     2035  1787    1787  ...            0.0            0.0        0.0   
...         ...     ...  ...            ...            ...        ...   
4947 2029     0       0  ...            0.0            0.0        0.0   
     2032     0       0  ...            0.0            0.0        0.0   
     2035     0       0  ...            0.0            0.0        0.0   
     2040     0       0  ...            0.0            0.0        0.0   
     2050     0       0  ...            0.0            0.0        0.0   

           infill_mf  infill_emp          dev        vac  unusable  vacancy  \
taz  year                                                                     
13   2022        0.0         0.0  4889.004243    0.00000       0.0       83   
     2026        0.0         0.0  4889.004243    0.00000       0.0       66   
     2029        0.0         0.0  4889.004243    0.00000       0.0       64   
     2032        0.0         0.0  4889.004243    0.00000       0.0       63   
     2035        0.0         0.0  4889.004243    0.00000       0.0       60   
...              ...         ...          ...        ...       ...      ...   
4947 2029        0.0         0.0   960.073851  270.55383       0.0        6   
     2032        0.0         0.0   960.073851  270.55383       0.0        8   
     2035        0.0         0.0   960.073851  270.55383       0.0        8   
     2040        0.0         0.0   960.073851  270.55383       0.0        8   
     2050        0.0         0.0   960.073851  270.55383       0.0        8   

           vacancy_rate  
taz  year                
13   2022      0.083585  
     2026      0.066465  
     2029      0.064451  
     2032      0.063444  
     2035      0.060423  
...                 ...  
4947 2029      0.750000  
     2032      1.000000  
     2035      1.000000  
     2040      1.000000  
     2050      1.000000  

[34545 rows x 125 columns]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.21s/it]


geotype  scenario     taz     luz       zip  \
jurisdiction year                                                     
Carlsbad     2022  jurisdiction         0  294881  126916  69743180   
             2026  jurisdiction         0  294881  126916  69743180   
             2029  jurisdiction         0  294881  126916  69743180   
             2032  jurisdiction         0  294881  126916  69743180   
             2035  jurisdiction         0  294881  126916  69743180   
...                         ...       ...     ...     ...       ...   
Vista        2029  jurisdiction         0  627759  135889  65655038   
             2032  jurisdiction         0  627759  135889  65655038   
             2035  jurisdiction         0  627759  135889  65655038   
             2040  jurisdiction         0  627759  135889  65655038   
             2050  jurisdiction         0  627759  135889  65655038   

                   pseudomsa  district27     pop     hhp    gq  ...  \
jurisdiction year                                               ...   
Carlsbad     2022       4550         859  115585  114619   966  ...   
             2026       4550         859  117853  116921   932  ...   
             2029       4550         859  121388  120450   938  ...   
             2032       4550         859  121713  120773   940  ...   
             2035       4550         859  123720  122767   953  ...   
...                      ...         ...     ...     ...   ...  ...   
Vista        2029       4928         713  101464   99458  2006  ...   
             2032       4928         713  102275  100263  2012  ...   
             2035       4928         713  104034  101976  2058  ...   
             2040       4928         713  105305  103261  2044  ...   
             2050       4928         713  104240  102356  1884  ...   

                   redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
jurisdiction year                                                       
Carlsbad     2022      48.981013     266.827288   2.975080  24.947334   
             2026      21.651900     240.779409   1.212635  12.087944   
             2029      15.077134     225.761543   0.000000  11.896798   
             2032       9.245455     212.204984   0.000000   0.000000   
             2035       1.491617     204.058286   0.000000   0.000000   
...                          ...            ...        ...        ...   
Vista        2029      90.628278     107.389934  10.431880   0.000000   
             2032      90.226848     106.556701   4.910551   0.000000   
             2035      89.908896     105.086247   0.000000   0.000000   
             2040      85.889377     103.759234   0.000000   0.000000   
             2050      85.889377      64.388351   0.000000   0.000000   

                   infill_emp           dev         vac     unusable  vacancy  \
jurisdiction year                                                               
Carlsbad     2022  627.031082  18724.597352  845.282086  5471.912863     3113   
             2026  595.960729  19054.612290  525.787043  5471.912863     3646   
             2029  536.986770  19158.010274  422.389059  5471.912863     3924   
             2032  479.737698  19170.574169  409.825165  5471.912863     3947   
             2035  425.903764  19220.800147  359.599186  5471.912863     3767   
...                       ...           ...         ...          ...      ...   
Vista        2029  487.146471  11040.145603  333.059170   539.573510     1651   
             2032  484.422395  11052.685378  320.519395   539.573510     1673   
             2035  480.802577  11069.661065  303.543707   539.573510     1716   
             2040  452.861250  11096.379180  276.825592   539.573510     2156   
             2050  368.918526  11167.652975  205.551797   539.573510     3350   

                   vacancy_rate  
jurisdiction year                
Carlsbad     2022      0.064714  
             2026      0.071671  
             2029      0.073854  
  

In [7]:
# consolidated files:
writer = pd.ExcelWriter(r'Data/mgra15_based_input_consolidated_{}.xlsx'.format(version), engine='xlsxwriter')
for geo, df in tqdm(geo_dfs.items()):
    display(df)
    df.reset_index().sort_values([geo, 'year']).to_excel(writer, sheet_name = '{}'.format(geo), index=False)
writer.save()

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

,,geotype,scenario,taz,luz,zip,pseudomsa,district27,pop,hhp,gq,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
mgra,year,,,,,,,,,,,,,,,,,,,,,
772,2022,mgra,0,4457,118,92021,5,13,6,6,0,...,0.0,0.0,0.0,0.000000,0.000000,1.586066,0.391226,0.0,0,0.000000
822,2022,mgra,0,36,161,92054,6,1,48,48,0,...,0.0,0.0,0.0,0.000000,0.000000,3.492520,0.539814,0.0,3,0.111111
836,2022,mgra,0,70,161,92054,6,1,49,49,0,...,0.0,0.0,0.0,0.278538,0.134806,4.100712,0.137066,0.0,3,0.107143
2308,2022,mgra,0,2640,8,92105,2,9,254,254,0,...,0.0,0.0,0.0,0.000000,0.000000,5.757694,0.157669,0.0,4,0.040000
2358,2022,mgra,0,959,33,92106,2,26,0,0,0,...,0.0,0.0,0.0,0.000000,0.000000,1.923376,0.000000,0.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22170,2050,mgra,0,4938,221,92004,8,14,15,15,0,...,0.0,0.0,0.0,0.000000,0.000000,548.006129,0.000000,0.0,14,0.608696
22172,2050,mgra,0,4890,221,92036,8,14,0,0,0,...,0.0,0.0,0.0,0.000000,0.000000,274.619237,0.000000,0.0,0,NaN
22206,2050,mgra,0,4905,224,92004,8,14,0,0,0,...,0.0,0.0,0.0,0.000000,0.000000,5.436640,0.000000,0.0,1,1.000000


 17%|█████████████▊                                                                     | 1/6 [03:16<16:24, 196.96s/it]

geotype  scenario       taz      luz         zip  pseudomsa  \
region    year                                                               
San Diego 2022  region         0  61885698  2834354  2238859007     111419   
          2026  region         0  61885698  2834354  2238859007     111419   
          2029  region         0  61885698  2834354  2238859007     111419   
          2032  region         0  61885698  2834354  2238859007     111419   
          2035  region         0  61885698  2834354  2238859007     111419   
          2040  region         0  61885698  2834354  2238859007     111419   
          2050  region         0  61885698  2834354  2238859007     111419   

                district27      pop      hhp      gq  ...  redev_emp_res  \
region    year                                        ...                  
San Diego 2022      232154  3287306  3170895  116411  ...    5751.485158   
          2026      232154  3302237  3188816  113421  ...    3923.499753   
          2029      232154  3334675  3220394  114281  ...    3686.796230   
          2032      232154  3373033  3257421  115612  ...    3575.591472   
          2035      232154  3404362  3287304  117058  ...    3404.907336   
          2040      232154  3432211  3315776  116435  ...    3054.843543   
          2050      232154  3400250  3285676  114574  ...    2902.186610   

                redev_emp_emp    infill_sf   infill_mf   infill_emp  \
region    year                                                        
San Diego 2022    9816.537780  5077.680910  689.775228  8820.060976   
          2026    9374.358484  3919.240695  457.586141  8654.583028   
          2029    8979.347826  3729.214992  406.626178  8417.827502   
          2032    8790.898273  3639.102629  354.508863  8217.821170   
          2035    8128.687578  3312.415162  315.666382  7833.609627   
          2040    7623.962261  3242.902270  223.030615  7515.209493   
          2050    6827.913177  3002.764201  108.114246  6712.540908   

                         dev           vac       unusable  vacancy  \
region    year                                                       
San Diego 2022  2.087814e+06  63080.562020  114791.146477    75170   
          2026  2.100048e+06  51085.984387  114791.146477    85846   
          2029  2.102949e+06  48186.029811  114791.146477    92735   
          2032  2.105195e+06  45940.224893  114791.146477    94882   
          2035  2.107488e+06  43648.218842  114791.146477    96868   
          2040  2.109871e+06  41270.628084  114791.146477    99898   
          2050  2.112782e+06  38364.859834  114791.146477   103918   

                vacancy_rate  
region    year                
San Diego 2022      0.060835  
          2026      0.066673  
          2029      0.070253  
          2032      0.070441  
          2035      0.070558  
          2040      0.070819  
          2050      0.072242  

[7 rows x 126 columns]

geotype  scenario     taz     luz       zip  \
jurisdiction year                                                     
Carlsbad     2022  jurisdiction         0  294881  126916  69743180   
             2026  jurisdiction         0  294881  126916  69743180   
             2029  jurisdiction         0  294881  126916  69743180   
             2032  jurisdiction         0  294881  126916  69743180   
             2035  jurisdiction         0  294881  126916  69743180   
...                         ...       ...     ...     ...       ...   
Vista        2029  jurisdiction         0  627759  135889  65655038   
             2032  jurisdiction         0  627759  135889  65655038   
             2035  jurisdiction         0  627759  135889  65655038   
             2040  jurisdiction         0  627759  135889  65655038   
             2050  jurisdiction         0  627759  135889  65655038   

                   pseudomsa  district27     pop     hhp    gq  ...  \
jurisdiction year                                               ...   
Carlsbad     2022       4550         859  115585  114619   966  ...   
             2026       4550         859  117853  116921   932  ...   
             2029       4550         859  121388  120450   938  ...   
             2032       4550         859  121713  120773   940  ...   
             2035       4550         859  123720  122767   953  ...   
...                      ...         ...     ...     ...   ...  ...   
Vista        2029       4928         713  101464   99458  2006  ...   
             2032       4928         713  102275  100263  2012  ...   
             2035       4928         713  104034  101976  2058  ...   
             2040       4928         713  105305  103261  2044  ...   
             2050       4928         713  104240  102356  1884  ...   

                   redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
jurisdiction year                                                       
Carlsbad     2022      48.981013     266.827288   2.975080  24.947334   
             2026      21.651900     240.779409   1.212635  12.087944   
             2029      15.077134     225.761543   0.000000  11.896798   
             2032       9.245455     212.204984   0.000000   0.000000   
             2035       1.491617     204.058286   0.000000   0.000000   
...                          ...            ...        ...        ...   
Vista        2029      90.628278     107.389934  10.431880   0.000000   
             2032      90.226848     106.556701   4.910551   0.000000   
             2035      89.908896     105.086247   0.000000   0.000000   
             2040      85.889377     103.759234   0.000000   0.000000   
             2050      85.889377      64.388351   0.000000   0.000000   

                   infill_emp           dev         vac     unusable  vacancy  \
jurisdiction year                                                               
Carlsbad     2022  627.031082  18724.597352  845.282086  5471.912863     3113   
             2026  595.960729  19054.612290  525.787043  5471.912863     3646   
             2029  536.986770  19158.010274  422.389059  5471.912863     3924   
             2032  479.737698  19170.574169  409.825165  5471.912863     3947   
             2035  425.903764  19220.800147  359.599186  5471.912863     3767   
...                       ...           ...         ...          ...      ...   
Vista        2029  487.146471  11040.145603  333.059170   539.573510     1651   
             2032  484.422395  11052.685378  320.519395   539.573510     1673   
             2035  480.802577  11069.661065  303.543707   539.573510     1716   
             2040  452.861250  11096.379180  276.825592   539.573510     2156   
             2050  368.918526  11167.652975  205.551797   539.573510     3350   

                   vacancy_rate  
jurisdiction year                
Carlsbad     2022      0.064714  
             2026      0.071671  
             2029      0.073854  
  

...

 67%|████████████████████████████████████████████████████████                            | 4/6 [03:17<01:07, 33.68s/it]

geotype  scenario     taz       zip  pseudomsa  district27    pop  \
luz year                                                                     
1   2022     luz         0  221777  13631580        296        3996   7233   
    2026     luz         0  221777  13631580        296        3996   7162   
    2029     luz         0  221777  13631580        296        3996   7155   
    2032     luz         0  221777  13631580        296        3996   7155   
    2035     luz         0  221777  13631580        296        3996   7273   
...          ...       ...     ...       ...        ...         ...    ...   
245 2029     luz         0  169137   3588811        195         507  10565   
    2032     luz         0  169137   3588811        195         507   9694   
    2035     luz         0  169137   3588811        195         507   9676   
    2040     luz         0  169137   3588811        195         507   9849   
    2050     luz         0  169137   3588811        195         507  10289   

            hhp  gq  gq_mil  ...  redev_emp_res  redev_emp_emp  infill_sf  \
luz year                     ...                                            
1   2022   7191  42       0  ...       0.115001      17.192360   0.000000   
    2026   7121  41       0  ...       0.000000      17.192360   0.000000   
    2029   7114  41       0  ...       0.000000      17.192360   0.000000   
    2032   7114  41       0  ...       0.000000      17.192360   0.000000   
    2035   7231  42       0  ...       0.000000      17.192360   0.000000   
...         ...  ..     ...  ...            ...            ...        ...   
245 2029  10497  68       0  ...       3.840816      93.776818  93.047038   
    2032   9625  69       0  ...       3.840816      92.615707  90.385080   
    2035   9606  70       0  ...       0.369939      88.284980  80.604189   
    2040   9780  69       0  ...       0.369939      84.820773  76.973511   
    2050  10228  61       0  ...       0.000000      52.019728  59.161195   

          infill_mf  infill_emp         dev        vac   unusable  vacancy  \
luz year                                                                     
1   2022   0.000000    6.912115  876.825496  19.026480  59.487941      188   
    2026   0.000000    6.912115  882.809483  14.190585  59.487941      211   
    2029   0.000000    6.912115  882.809483  14.190585  59.487941      192   
    2032   0.000000    6.912115  882.809483  14.190585  59.487941      179   
    2035   0.000000    6.912115  882.809483  14.190585  59.487941      114   
...             ...         ...         ...        ...        ...      ...   
245 2029  22.499032    8.162603  689.161390  17.742480   0.000000      232   
    2032   6.587490    8.162603  691.953903  14.949968   0.000000      713   
    2035   0.000000    8.162603  694.461223  12.442648   0.000000      989   
    2040   0.000000    8.162603  695.031630  11.872241   0.000000      893   
    2050   0.000000    8.162603  698.477564   8.426307   0.000000      743   

          vacancy_rate  
luz year                
1   2022      0.049292  
    2026      0.054409  
    2029      0.049446  
    2032      0.045921  
    2035      0.029096  
...                ...  
245 2029      0.057654  
    2032      0.168717  
    2035      0.218419  
    2040      0.195619  
    2050      0.158456  

[1715 rows x 125 columns]

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [03:19<00:23, 23.12s/it]

geotype  scenario   luz     zip  pseudomsa  district27   pop   hhp  \
taz  year                                                                      
13   2022     taz         0   676  370071         24           4  4646  2859   
     2026     taz         0   676  370071         24           4  4570  2816   
     2029     taz         0   676  370071         24           4  4549  2788   
     2032     taz         0   676  370071         24           4  4535  2760   
     2035     taz         0   676  370071         24           4  4528  2741   
...           ...       ...   ...     ...        ...         ...   ...   ...   
4947 2029     taz         0  1808  735472         64         112     5     5   
     2032     taz         0  1808  735472         64         112     0     0   
     2035     taz         0  1808  735472         64         112     0     0   
     2040     taz         0  1808  735472         64         112     0     0   
     2050     taz         0  1808  735472         64         112     0     0   

             gq  gq_mil  ...  redev_emp_res  redev_emp_emp  infill_sf  \
taz  year                ...                                            
13   2022  1787    1787  ...            0.0            0.0        0.0   
     2026  1754    1754  ...            0.0            0.0        0.0   
     2029  1761    1761  ...            0.0            0.0        0.0   
     2032  1775    1775  ...            0.0            0.0        0.0   
     2035  1787    1787  ...            0.0            0.0        0.0   
...         ...     ...  ...            ...            ...        ...   
4947 2029     0       0  ...            0.0            0.0        0.0   
     2032     0       0  ...            0.0            0.0        0.0   
     2035     0       0  ...            0.0            0.0        0.0   
     2040     0       0  ...            0.0            0.0        0.0   
     2050     0       0  ...            0.0            0.0        0.0   

           infill_mf  infill_emp          dev        vac  unusable  vacancy  \
taz  year                                                                     
13   2022        0.0         0.0  4889.004243    0.00000       0.0       83   
     2026        0.0         0.0  4889.004243    0.00000       0.0       66   
     2029        0.0         0.0  4889.004243    0.00000       0.0       64   
     2032        0.0         0.0  4889.004243    0.00000       0.0       63   
     2035        0.0         0.0  4889.004243    0.00000       0.0       60   
...              ...         ...          ...        ...       ...      ...   
4947 2029        0.0         0.0   960.073851  270.55383       0.0        6   
     2032        0.0         0.0   960.073851  270.55383       0.0        8   
     2035        0.0         0.0   960.073851  270.55383       0.0        8   
     2040        0.0         0.0   960.073851  270.55383       0.0        8   
     2050        0.0         0.0   960.073851  270.55383       0.0        8   

           vacancy_rate  
taz  year                
13   2022      0.083585  
     2026      0.066465  
     2029      0.064451  
     2032      0.063444  
     2035      0.060423  
...                 ...  
4947 2029      0.750000  
     2032      1.000000  
     2035      1.000000  
     2040      1.000000  
     2050      1.000000  

[34545 rows x 125 columns]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:56<00:00, 39.36s/it]


In [8]:
# write year data into excel
for year in tqdm(years):
    writer = pd.ExcelWriter(r'Data/{}/mgra15_based_input_{}_{}.xlsx'.format(year, year, version), engine='xlsxwriter')
    for geo, df in tqdm(geo_dfs.items()):
        output = df.reset_index()
        output = output[output['year'].isin([year])].reset_index(drop=True)
        if 'taz' in list(output.columns)and geo != 'taz':
            output = output.drop('taz', axis=1)
        if 'LUZ' in list(output.columns):
            output = output.drop('LUZ', axis=1)
        output.insert(1, 'geozone', output[geo])
        display(output)
        output.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

,mgra,geozone,year,geotype,scenario,luz,zip,pseudomsa,district27,pop,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,772,772,2022,mgra,0,118,92021,5,13,6,...,0.0,0.0,0.0,0.000000,0.000000,1.586066,0.391226,0.000000,0,0.000000
1,822,822,2022,mgra,0,161,92054,6,1,48,...,0.0,0.0,0.0,0.000000,0.000000,3.492520,0.539814,0.000000,3,0.111111
2,836,836,2022,mgra,0,161,92054,6,1,49,...,0.0,0.0,0.0,0.278538,0.134806,4.100712,0.137066,0.000000,3,0.107143
3,2308,2308,2022,mgra,0,8,92105,2,9,254,...,0.0,0.0,0.0,0.000000,0.000000,5.757694,0.157669,0.000000,4,0.040000
4,2358,2358,2022,mgra,0,33,92106,2,26,0,...,0.0,0.0,0.0,0.000000,0.000000,1.923376,0.000000,0.000000,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24316,3820,3820,2022,mgra,0,85,91913,4,12,435,...,0.0,0.0,0.0,0.000000,0.000000,19.227201,0.000000,0.000000,0,0.000000
24317,3825,3825,2022,mgra,0,19,91910,4,12,435,...,0.0,0.0,0.0,0.000000,0.000000,27.287517,0.000000,0.000000,2,0.014085
24318,3702,3702,2022,mgra,0,58,92129,3,4,307,...,0.0,0.0,0.0,0.000000,0.000000,24.154122,2.508526,30.232698,3,0.026786
24319,3804,3804,2022,mgra,0,90,92154,4,11,143,...,0.0,0.0,0.0,0.000000,0.398702,3.433450,0.822900,0.000000,0,0.000000



 17%|██████████████                                                                      | 1/6 [00:26<02:11, 26.29s/it]

,region,geozone,year,geotype,scenario,luz,zip,pseudomsa,district27,pop,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,San Diego,San Diego,2022,region,0,2834354,2238859007,111419,232154,3287306,...,5751.485158,9816.53778,5077.68091,689.775228,8820.060976,2.087814e+06,63080.56202,114791.146477,75170,0.060835


,jurisdiction,geozone,year,geotype,scenario,luz,zip,pseudomsa,district27,pop,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,Carlsbad,Carlsbad,2022,jurisdiction,0,126916,69743180,4550,859,115585,...,48.981013,266.827288,2.975080,24.947334,627.031082,1.872460e+04,845.282086,5471.912863,3113,0.064714
1,Chula Vista,Chula Vista,2022,jurisdiction,0,112956,98805269,4299,12401,276785,...,245.136647,319.726038,0.503478,8.292870,499.112329,2.774602e+04,2730.834133,3057.663242,2731,0.030984
2,Coronado,Coronado,2022,jurisdiction,0,23634,20542192,446,2243,22277,...,2.190672,122.140115,9.515815,0.508823,26.040527,9.757778e+03,24.952730,309.573962,2210,0.228660
3,Del Mar,Del Mar,2022,jurisdiction,0,2904,6072924,198,148,3929,...,3.042198,9.985388,0.000000,0.000000,0.378786,1.000006e+03,33.310773,102.908124,655,0.249144
4,El Cajon,El Cajon,2022,jurisdiction,0,82262,54107791,2940,7644,105638,...,23.834850,63.628054,2.520019,4.246081,376.072664,8.846823e+03,272.873824,155.789973,1194,0.032632
5,Encinitas,Encinitas,2022,jurisdiction,0,123345,75548908,4926,1642,61515,...,9.670661,52.607779,1.557644,0.000000,162.988627,9.839521e+03,426.629127,2246.207532,2068,0.077555
6,Escondido,Escondido,2022,jurisdiction,0,192371,86504804,6580,2820,150679,...,64.587804,129.064203,5.776852,0.000000,560.361195,1.724417e+04,1665.539483,5034.396925,1810,0.036566
7,Imperial Beach,Imperial Beach,2022,jurisdiction,0,29492,13973664,608,1672,26243,...,3.533559,46.196002,0.130431,0.000000,10.468602,1.733914e+03,11.617292,1063.725145,714,0.071158
8,La Mesa,La Mesa,2022,jurisdiction,0,51501,43120619,2345,4066,60472,...,12.180635,6.519877,0.527566,0.000000,142.468202,5.676175e+03,122.549718,3.244729,1229,0.046792
9,Lemon Grove,Lemon Grove,2022,jurisdiction,0,11559,16550100,879,1806,27242,...,12.998008,1.001917,1.646096,0.000000,34.942079,2.384757e+03,63.252898,0.048723,337,0.036686


...

,cpa,geozone,year,geotype,scenario,luz,zip,pseudomsa,district27,pop,...,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate,cpa_name
0,0,0,2050,cpa,0,1343072,831452093,47673,49808,1489936,...,1112.688656,1.882467,96.361037,2944.742320,185541.534648,6698.644894,31892.368304,42192,0.069470,Not in CPA
1,1401,1401,2050,cpa,0,841,2670998,58,270,409,...,0.000000,0.000000,0.000000,68.980657,1257.673157,29.806011,0.000000,0,0.000000,Balboa Park
2,1402,1402,2050,cpa,0,1291,5158328,112,715,4668,...,15.106046,0.000000,0.092226,30.680444,558.557130,17.423924,0.013872,1308,0.447485,Barrio Logan
3,1403,1403,2050,cpa,0,9179,6172576,201,268,14039,...,6.490821,0.000000,0.000000,16.490329,1487.152519,0.256000,78.723801,85,0.013554,Carmel Mountain Ranch
4,1404,1404,2050,cpa,0,9564,31498626,366,5163,57250,...,104.073145,0.000000,0.117746,59.381235,1443.358050,67.840469,0.043054,1767,0.046604,Downtown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1953,1953,2050,cpa,0,88160,47302443,3585,1398,47680,...,0.000000,0.000000,0.000000,0.000000,25642.878265,0.000000,5896.458971,1258,0.065040,Fallbrook
84,1954,1954,2050,cpa,0,8180,4328274,288,47,36292,...,0.000000,0.000000,0.000000,0.000000,150245.997039,0.000000,34.539931,528,0.069712,Pendleton-De Luz
85,1955,1955,2050,cpa,0,18725,9846996,749,1381,1757,...,0.000000,0.000000,0.000000,0.000000,5366.997447,0.000000,589.345390,39,0.050847,Rainbow
86,1998,1998,2050,cpa,0,895,644305,35,98,808,...,0.000000,0.000000,0.000000,0.000000,2620.091211,0.000000,41.454752,0,0.000000,Barona


,luz,geozone,year,geotype,scenario,zip,pseudomsa,district27,pop,hhp,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,1,1,2050,luz,0,13631580,296,3996,7220,7181,...,0.000000,17.192360,0.000000,0.000000,6.912115,882.847867,14.152201,59.487941,77,0.019361
1,2,2,2050,luz,0,20999350,456,6156,17594,16910,...,3.979863,0.792277,0.000000,0.119559,2.962959,875.573627,16.536648,22.277557,173,0.015625
2,3,3,2050,luz,0,12802577,278,2404,18383,17858,...,1.587928,2.121930,0.000000,0.201197,11.100265,1019.125578,18.246235,68.672775,78,0.006908
3,4,4,2050,luz,0,22842392,496,1984,37623,37542,...,0.567818,0.000000,0.000000,0.160697,0.000000,1485.383579,6.325351,5.674370,78,0.003397
4,5,5,2050,luz,0,11881946,386,875,38002,37767,...,7.112804,64.305169,0.000000,0.000000,205.765814,2931.739346,202.357129,84.335003,124,0.005893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,241,241,2050,luz,0,10129220,770,110,15591,15469,...,0.000000,2.169281,0.000000,0.000000,8.935435,1760.262128,34.546466,28.649474,285,0.048660
241,242,242,2050,luz,0,1472592,48,267,2789,2789,...,0.000000,34.226770,0.000000,0.000000,36.880099,387.804901,27.248322,0.000000,6,0.004304
242,243,243,2050,luz,0,2487267,81,432,1622,1622,...,0.000000,60.211690,0.000000,0.000000,74.380437,788.405251,37.026398,71.734428,0,0.000000
243,244,244,2050,luz,0,4145477,135,720,10950,10950,...,0.000000,9.687128,0.000000,0.000000,68.406062,674.435744,5.670487,19.166255,1829,0.254416



 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [00:32<00:04,  4.15s/it]

,taz,geozone,year,geotype,scenario,luz,zip,pseudomsa,district27,pop,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,13,13,2050,taz,0,676,370071,24,4,4368,...,0.0,0.000000,0.0,0.000000,0.000000,4889.004243,0.000000,0.000000,60,0.060423
1,14,14,2050,taz,0,1521,828495,54,9,1745,...,0.0,0.000000,0.0,0.000000,0.000000,25603.832503,0.000000,0.000000,104,0.414343
2,15,15,2050,taz,0,720,276165,18,3,4282,...,0.0,0.000000,0.0,0.000000,0.000000,1108.992996,0.000000,0.000000,5,0.016556
3,16,16,2050,taz,0,483,276162,18,3,56,...,0.0,0.000000,0.0,1.208995,1.562854,122.196625,0.023044,9.290139,30,0.447761
4,17,17,2050,taz,0,2873,1564935,102,17,10562,...,0.0,0.000000,0.0,0.000000,0.000000,73140.455309,0.000000,0.000000,218,0.135657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930,4943,4943,2050,taz,0,1350,552235,48,84,142,...,0.0,459.067789,0.0,0.000000,0.000000,94142.693533,0.000000,0.000000,0,0.000000
4931,4944,4944,2050,taz,0,904,367736,32,56,314,...,0.0,4.215455,0.0,0.000000,0.000000,364.012742,43.630083,0.000000,30,0.179641
4932,4945,4945,2050,taz,0,452,183868,16,28,209,...,0.0,0.000000,0.0,0.000000,0.000000,427.573564,578.280406,0.000000,27,0.230769
4933,4946,4946,2050,taz,0,884,368016,32,56,0,...,0.0,0.000000,0.0,0.000000,0.000000,35300.498583,0.000000,0.000000,0,NaN



100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [06:36<00:00, 56.65s/it]


### 2026 and 2029 Diff file- Series 14 (Jurisdiction, CPA, LUZ, Region) 

In [12]:
# get series 14 data
series14_df = {}

for geo in tqdm(geo_dfs.keys()):
    if geo == 'mgra':
        series14_df['mgra'] = mgra_output('41', False)
        series14_df['mgra']['vacancy'] = series14_df['mgra']['hs'] - series14_df['mgra']['hh']
        series14_df['mgra']['vacancy_rate'] = series14_df['mgra']['vacancy']/series14_df['mgra']['hs']
    else:
        sr14 = series14_df['mgra'].copy()
        sr14 = sr14[sr14['year'].astype(int)>2025]
        series14_df[geo] = create_mgra_denorm_table(mgra_denorm_path=sr14, 
                                         geo_level=geo, 
                                         output_folder_path=False, 
                                         version='SR14',
                                         series=14)
    if geo == 'cpa':
        series14_df[geo] = add_cpa_names(series14_df[geo].reset_index(), 14).set_index(['cpa', 'year'])
        
    series14_df[geo].insert(0, 'geotype', geo)

series14_df['cpa']

 50%|██████████████████████████████████████████                                          | 3/6 [00:24<00:19,  6.59s/it]C:\Users\jchu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):
100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:36<00:00,  6.06s/it]


geotype  index        taz       hs   hs_sf    hs_mf  hs_mh  \
cpa    year                                                              
1442.0 2026     cpa  280.0  1633143.0  26556.0  7861.0  18695.0    0.0   
       2029     cpa  281.0  1633143.0  28224.0  7892.0  20332.0    0.0   
       2030     cpa  282.0  1633143.0  28794.0  7892.0  20902.0    0.0   
       2032     cpa  283.0  1633143.0  29056.0  7896.0  21160.0    0.0   
       2035     cpa  284.0  1633143.0  29523.0  7897.0  21626.0    0.0   
...             ...    ...        ...      ...     ...      ...    ...   
1906.0 2032     cpa  531.0   622253.0   3383.0  3251.0     36.0   96.0   
       2035     cpa  532.0   622253.0   3474.0  3342.0     36.0   96.0   
       2040     cpa  533.0   622253.0   3554.0  3419.0     39.0   96.0   
       2045     cpa  534.0   622253.0   3593.0  3458.0     39.0   96.0   
       2050     cpa  535.0   622253.0   3640.0  3505.0     39.0   96.0   

                  hh   hh_sf    hh_mf  ...  milestocoast          acres  \
cpa    year                            ...                                
1442.0 2026  24968.0  7570.0  17398.0  ...     1941.9361    2684.380125   
       2029  26460.0  7619.0  18841.0  ...     1941.9361    2684.380125   
       2030  26966.0  7629.0  19337.0  ...     1941.9361    2684.380125   
       2032  27266.0  7632.0  19634.0  ...     1941.9361    2684.380125   
       2035  27637.0  7632.0  20005.0  ...     1941.9361    2684.380125   
...              ...     ...      ...  ...           ...            ...   
1906.0 2032   3236.0  3109.0     36.0  ...     3096.6867  107365.932568   
       2035   3308.0  3181.0     36.0  ...     3096.6867  107365.932568   
       2040   3370.0  3242.0     37.0  ...     3096.6867  107365.932568   
       2045   3415.0  3286.0     38.0  ...     3096.6867  107365.932568   
       2050   3456.0  3326.0     39.0  ...     3096.6867  107365.932568   

             effective_acres     land_acres  MicroAccessTime  remoteAVParking  \
cpa    year                                                                     
1442.0 2026      2640.743833    2684.380125           4600.0              2.0   
       2029      2640.743833    2684.380125           4600.0              2.0   
       2030      2640.743833    2684.380125           4600.0              2.0   
       2032      2640.743833    2684.380125           4600.0              2.0   
       2035      2640.743833    2684.380125           4600.0              2.0   
...                      ...            ...              ...              ...   
1906.0 2032     51550.884299  106651.799938          18960.0              0.0   
       2035     51550.884299  106651.799938          18960.0              0.0   
       2040     51550.884299  106651.799938          18960.0              0.0   
       2045     51550.884299  106651.799938          18960.0              0.0   
       2050     51550.884299  106651.799938          18960.0              0.0   

             refueling_stations  vacancy  vacancy_rate       cpa_name  
cpa    year                                                            
1442.0 2026                 9.0   1588.0      0.059798         Uptown  
       2029                 9.0   1764.0      0.062500         Uptown  
       2030                 9.0   1828.0      0.063485         Uptown  
       2032                 9.0   1790.0      0.061605         Uptown  
       2035                 9.0   1886.0      0.063882         Uptown  
...                         ...      ...           ...            ...  
1906.0 2032                 1.0    147.0      0.043453  Jamul-Dulzura  
       2035                 1.0    166.0      0.047784  Jamul-Dulzura  
       2040                 1.0    184.0      0.051773  Jamul-Dulzura  
       2045                 1.0    178.0      0.049541  Jamul-Dulzura  
       2050                 1.0    184.0      0.050549  Jamul-Dulzura  

[697 rows x 111 columns]

In [13]:
# create df of all differences
def full_diff_pivot(df1, df2, geo, series14 = False):
    
    if 'taz' in list(df1.columns) and geo != 'taz':
        df1 = df1.drop('taz', axis=1)
    if 'LUZ' in list(df1.columns):
        df1 = df1.drop('LUZ', axis=1)
        
    if 'taz' in list(df2.columns) and geo != 'taz':
        df2 = df2.drop('taz', axis=1)
    if 'LUZ' in list(df2.columns):
        df2 = df2.drop('LUZ', axis=1)
    
    # attributes matching
    current_headers = common_values(df1, df2)

    correct_headers = pd.read_csv(
        rf"T:/socioec/Current_Projects/SR15/S0/version6/abm_csv/ForQApurposes/mgra15_based_input_2026_01.csv", nrows=0).columns

    attributes = [header for header in correct_headers if header in current_headers] + ['vacancy', 'vacancy_rate']
    
    if geo not in attributes:
        attributes = [geo] + attributes
    
    # create diffs
    df1_value_name = df1['year'].iloc[0]
    df2_value_name = df2['year'].iloc[0]
    num_years = abs(int(df2_value_name) - int(df1_value_name))
    
    if series14 == 'df2':
        df2_value_name = df2_value_name + ' SR14'
    elif series14 == 'df1':
        df1_value_name = df1_value_name + ' SR14'
    
    df1_pivot = df1[attributes].melt(id_vars=[geo],
                                               var_name = 'attribute',
                                               value_name = df1_value_name)
    df2_pivot = df2[attributes].melt(id_vars=[geo],
                                               var_name = 'attribute',
                                               value_name = df2_value_name)
    full_diff_df =  df1_pivot.merge(df2_pivot,
                                    how='outer',
                                    on=[geo, 'attribute'])
    full_diff_df['diff'] = full_diff_df[df1_value_name] - full_diff_df[df2_value_name]
    full_diff_df['pct diff'] = full_diff_df['diff']/full_diff_df[df2_value_name]
    
    # create CAGRs
    if num_years != 0:
        
        full_diff_df['CAGR'] = ((full_diff_df[df1_value_name]/full_diff_df[df2_value_name])**(1/num_years))-1
        
        # add in thresholds column
        threshold_dict = {'mgra': (350, .03), 'cpa': (500, .03), 'luz': (700, .03)}
        if geo in threshold_dict.keys():
            full_diff_df['threshold_passed'] = (full_diff_df['diff'] < threshold_dict[geo][0]) | (full_diff_df['CAGR'] < threshold_dict[geo][1])
    
    # merging the dataframes together
    full_diff_df = full_diff_df.merge(
        full_diff_df.groupby(['attribute']).sum().reset_index()[['attribute', df1_value_name, df2_value_name]],
        how='outer',
        on=['attribute'],
        suffixes=('',' Share'))
    full_diff_df[df1_value_name + ' Share'] = (full_diff_df[df1_value_name]/full_diff_df[df1_value_name + ' Share'])
    full_diff_df[df2_value_name + ' Share'] = (full_diff_df[df2_value_name]/full_diff_df[df2_value_name + ' Share'])
    full_diff_df['Geographic Share Change'] = full_diff_df[df1_value_name + ' Share'] - full_diff_df[df2_value_name + ' Share']
    
    full_diff_df.insert(0, 'geotype', geo)
    full_diff_df.insert(1, 'geozone', full_diff_df[geo])
    
    return full_diff_df

In [14]:
geo='cpa'
full_diff_pivot(geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == '2026'],
                   geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == '2022'],
                   geo
                   )

,geotype,geozone,cpa,attribute,2026,2022,diff,pct diff,CAGR,threshold_passed,2026 Share,2022 Share,Geographic Share Change
0,cpa,0,0,pop,1.408743e+06,1.399346e+06,9397.000000,0.006715,0.001675,True,0.426603,0.425682,0.000921
1,cpa,1401,1401,pop,4.090000e+02,4.130000e+02,-4.000000,-0.009685,-0.002430,True,0.000124,0.000126,-0.000002
2,cpa,1402,1402,pop,4.185000e+03,3.885000e+03,300.000000,0.077220,0.018770,True,0.001267,0.001182,0.000086
3,cpa,1403,1403,pop,1.474000e+04,1.229700e+04,2443.000000,0.198666,0.046344,False,0.004464,0.003741,0.000723
4,cpa,1404,1404,pop,5.106200e+04,4.789600e+04,3166.000000,0.066102,0.016131,True,0.015463,0.014570,0.000893
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,cpa,1953,1953,vacancy_rate,5.161505e-02,4.935834e-02,0.002257,0.045721,0.011239,True,0.005642,0.004774,0.000867
5364,cpa,1954,1954,vacancy_rate,8.376272e-02,8.270577e-02,0.001057,0.012780,0.003180,True,0.009155,0.008000,0.001155
5365,cpa,1955,1955,vacancy_rate,9.366755e-02,9.247028e-02,0.001197,0.012948,0.003221,True,0.010238,0.008945,0.001293
5366,cpa,1998,1998,vacancy_rate,3.472222e-03,3.472222e-03,0.000000,0.000000,0.000000,True,0.000380,0.000336,0.000044


In [16]:
# adding in to excel files
for i in tqdm(range(1, len(years))):
    writer = pd.ExcelWriter(r'Results/mgra15_based_input_{}_{}_minus_SR14.xlsx'.format(version, years[i]), engine='xlsxwriter')
    for geo in tqdm(geo_dfs.keys()):
        df = full_diff_pivot(geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == years[i]],
                       series14_df[geo].reset_index()[series14_df[geo].reset_index()['year'] == years[i]],
                       geo,
                       series14 = 'df2'
                       )

        if geo == 'cpa':
            df = add_cpa_names(df, 15).rename(columns={'cpa_name':'cpa_name_SR15'})
            df = add_cpa_names(df, 14).rename(columns={'cpa_name':'cpa_name_SR14'})

        df.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()
    
    writer = pd.ExcelWriter(r'Results/mgra15_based_input_{}_{}_minus_{}.xlsx'.format(version, years[i], years[i-1]), engine='xlsxwriter')

    for geo in tqdm(geo_dfs.keys()):
        df = full_diff_pivot(geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == years[i]],
                       geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == years[i-1]],
                       geo
                       )

        if geo == 'mgra':
            df.to_csv(r'Results/mgra15_based_input_{}_{}_minus_{}_{}.csv'.format(version, years[i], years[i-1], geo))
        else:
            df.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:47<00:00, 18.00s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:03<00:00, 20.54s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:49<00:00, 18.21s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:49<00:00, 18.29s/it]

 67%|████████████████████████████████████████████████████████                            | 4/6 [01:31<00:26, 13.19s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:49<00:00, 18.27s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:33<00:00, 15.52s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [24:05<00:00, 240.85s/it]


### Diff columns for hhi and emp

In [17]:
# employment
pattern = re.compile("^emp_")
emp_columns = [col for col in geo_dfs['mgra'].columns if pattern.match(col)]

# household income
pattern = re.compile("i\d+")
household_income_columns = [col for col in geo_dfs['mgra'].columns if pattern.match(col)]

In [18]:
def diff_columns(df, geo, columns, col_var_name, value_name, first_year, second_year):
    def column_pct(df, geo, columns, year, col_var_name):
        pct_df = df[df['year'] == year][[geo] + columns].set_index(geo)
        return (pct_df.T/pct_df.sum(axis=1)).T.reset_index().melt(id_vars = [geo], var_name = col_var_name, value_name=value_name)

    df_values_1 = df[df['year']==first_year][[geo]+columns].melt(id_vars = [geo], var_name = col_var_name, value_name=first_year)
    df1 = column_pct(df, geo, columns, first_year, col_var_name)
    
    df_values_2 = df[df['year']==second_year][[geo]+columns].melt(id_vars = [geo], var_name = col_var_name, value_name=second_year)
    df2 = column_pct(df, geo, columns, second_year, col_var_name)

    output = df_values_1.merge(df_values_2, how='outer', on=[geo, col_var_name])
    output['diff'] = output[first_year] - output[second_year]
    output['pct_diff'] = output['diff']/output[second_year]
    
    num_years = int(first_year) - int(second_year)
    output['CAGR'] = ((output[first_year]/output[second_year])**(1/num_years))-1
    
    output = output.merge(df1, how='outer', on=[geo, col_var_name])
    output = output.merge(df2, how='outer', on=[geo, col_var_name], suffixes=(' '+first_year, ' '+second_year))
    output[value_name + ' Change'] = output[value_name + ' ' + first_year] - output[value_name + ' ' + second_year]
    
    output.insert(0, 'geotype', geo)
    output.insert(1, 'geozone', output[geo])
    return output

In [20]:
for i in tqdm(range(1, len(years))):
    writer = pd.ExcelWriter(r'Results/houeshold_income_share_diffs_{}_{}_minus_{}.xlsx'.format(version, years[i], years[i-1]), engine='xlsxwriter')
    for geo in tqdm(geo_dfs.keys()):
        diff = diff_columns(geo_dfs[geo].reset_index(), geo, household_income_columns, 'hhi', 'Income Share', years[i], years[i-1])
        diff.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:54<00:00, 39.14s/it]


In [21]:
for i in tqdm(range(1, len(years))):
    writer = pd.ExcelWriter(r'Results/sector_share_diffs_{}_{}_minus_{}.xlsx'.format(version, years[i], years[i-1]), engine='xlsxwriter')
    for geo in tqdm(geo_dfs.keys()):
        diff = diff_columns(geo_dfs[geo].reset_index(), geo, household_income_columns, 'hhi', 'Sector Share', years[i], years[i-1])
        diff.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [04:01<00:00, 40.31s/it]


### Consolidated CGAR

In [22]:
geo_dfs['mgra'].reset_index()\
                                    .sort_values(['mgra', 'year'])\
                                    .set_index(['mgra', 'year'])\
                                    .select_dtypes(include='number')

scenario   taz  luz    zip  pseudomsa  district27  pop  hhp  gq  \
mgra  year                                                                    
1     2022         0  3010   10  92105          2           9  440  440   0   
      2026         0  3010   10  92105          2           9  411  411   0   
      2029         0  3010   10  92105          2           9  406  406   0   
      2032         0  3010   10  92105          2           9  406  406   0   
      2035         0  3010   10  92105          2           9  406  406   0   
...              ...   ...  ...    ...        ...         ...  ...  ...  ..   
24321 2029         0  3039  200  92026          7           3   55   55   0   
      2032         0  3039  200  92026          7           3   54   54   0   
      2035         0  3039  200  92026          7           3   53   53   0   
      2040         0  3039  200  92026          7           3   52   52   0   
      2050         0  3039  200  92026          7           3   51   51   0   

            gq_mil  ...  redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
mgra  year          ...                                                       
1     2022       0  ...            0.0            0.0        0.0        0.0   
      2026       0  ...            0.0            0.0        0.0        0.0   
      2029       0  ...            0.0            0.0        0.0        0.0   
      2032       0  ...            0.0            0.0        0.0        0.0   
      2035       0  ...            0.0            0.0        0.0        0.0   
...            ...  ...            ...            ...        ...        ...   
24321 2029       0  ...            0.0            0.0        0.0        0.0   
      2032       0  ...            0.0            0.0        0.0        0.0   
      2035       0  ...            0.0            0.0        0.0        0.0   
      2040       0  ...            0.0            0.0        0.0        0.0   
      2050       0  ...            0.0            0.0        0.0        0.0   

            infill_emp        dev      vac  unusable  vacancy  vacancy_rate  
mgra  year                                                                   
1     2022         0.0  18.837695  0.00000       0.0        2      0.011364  
      2026         0.0  18.837695  0.00000       0.0        8      0.045455  
      2029         0.0  18.837695  0.00000       0.0        8      0.045455  
      2032         0.0  18.837695  0.00000       0.0        6      0.034091  
      2035         0.0  18.837695  0.00000       0.0        4      0.022727  
...                ...        ...      ...       ...      ...           ...  
24321 2029         0.0   7.849918  0.43073       0.0        3      0.130435  
      2032         0.0   7.849918  0.43073       0.0        3      0.130435  
      2035         0.0   7.849918  0.43073       0.0        3      0.130435  
      2040         0.0   7.849918  0.43073       0.0        3      0.130435  
      2050         0.0   7.849918  0.43073       0.0        3      0.130435  

[170247 rows x 125 columns]

In [23]:
writer = pd.ExcelWriter(r'Data/mgra15_based_input_consolidated_CGAR_{}.xlsx'.format(version), engine='xlsxwriter')

for geo in tqdm(geo_dfs.keys()):
    consolidated_cgar = geo_dfs[geo].reset_index()\
                                        .sort_values([geo, 'year'])\
                                        .set_index([geo, 'year'])\
                                        .select_dtypes(include='number')

    year_diffs = consolidated_cgar.reset_index()
    year_diffs = year_diffs.assign(year_diff = year_diffs['year'].astype(int) - year_diffs['year'].astype(int).shift(1))
    year_diffs = year_diffs.assign(year = year_diffs['year'].astype(str)).set_index([geo, 'year'])

    cgar = consolidated_cgar.div(consolidated_cgar.shift(1))
    cgar = ((cgar.T.pow(1/year_diffs['year_diff']).T)-1).reset_index()
    
    cgar[cgar['year']!='2022'].sort_values([geo, 'year']).to_excel(writer, sheet_name = '{}'.format(geo), index=False)
writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:19<00:00, 23.25s/it]


### Negative values (Region, Jurisdiction, CPA, LUZ, MGRA)

In [26]:
for geo, df in geo_dfs.items():
    df = df.reset_index()[df.reset_index()['year'] != '2022']
    print('------------------------------------------------------------------')
    print(geo, df[(df.select_dtypes(include='number') < 0).any(axis=1)].shape)
    print('******************************************************************')
    display(df.select_dtypes(include='number').loc[:,(df.select_dtypes(include='number') < 0).any()])

------------------------------------------------------------------
mgra (0, 128)
******************************************************************


""
24321
24322
24323
24324
24325
...
170242
170243
170244
170245


------------------------------------------------------------------
region (0, 128)
******************************************************************


""
1
2
3
4
5
6


------------------------------------------------------------------
jurisdiction (0, 128)
******************************************************************


...

""
1
2
3
4
5
...
611
612
613
614


------------------------------------------------------------------
luz (0, 127)
******************************************************************


""
1
2
3
4
5
...
1710
1711
1712
1713


------------------------------------------------------------------
taz (0, 127)
******************************************************************


""
1
2
3
4
5
...
34540
34541
34542
34543


### Zero Values (Region, Jurisdiction, CPA, LUZ) 

In [27]:
for geo, df in geo_dfs.items():
    df = df.reset_index()[df.reset_index()['year'] != '2022']
    print('------------------------------------------------------------------')
    print(geo, df[(df.select_dtypes(include='number') == 0).any(axis=1)].shape)
    print('******************************************************************')
    display(df.select_dtypes(include='number').loc[:,(df.select_dtypes(include='number') == 0).any()])

------------------------------------------------------------------
mgra (145926, 128)
******************************************************************


,scenario,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,hhs,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
24321,0,24,24,0,0,0,0,0,12,2.400000,...,0.0,0.0,0.0,0.0,0.000000,4.812648,0.000000,0.0,2,0.166667
24322,0,0,0,0,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0.0,1.658937,1.658937,0.000000,0.0,0,NaN
24323,0,36,36,0,0,0,0,0,13,3.272727,...,0.0,0.0,0.0,0.0,0.000000,2.344121,0.000000,0.0,2,0.153846
24324,0,149,149,0,0,0,0,0,61,2.660714,...,0.0,0.0,0.0,0.0,0.000000,4.907733,3.105858,0.0,4,0.066667
24325,0,0,0,0,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.911417,3.296175,0.912689,0.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170242,0,15,15,0,0,0,0,0,11,1.666667,...,0.0,0.0,0.0,0.0,0.000000,548.006129,0.000000,0.0,14,0.608696
170243,0,0,0,0,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,274.619237,0.000000,0.0,0,NaN
170244,0,0,0,0,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,5.436640,0.000000,0.0,1,1.000000
170245,0,0,0,0,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,7.733600,0.000000,0.0,3,1.000000


------------------------------------------------------------------
region (6, 128)
******************************************************************


,scenario
1,0
2,0
3,0
4,0
5,0
6,0


------------------------------------------------------------------
jurisdiction (114, 128)
******************************************************************


...

,cpa,scenario,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
1,0,0,1408743,1382603,26140,9094,17046,1764,15282,672924,...,701.504051,1656.462977,32.317803,388.410318,4325.568559,180359.009413,11871.831508,31892.368304,29934,0.056963
2,0,0,1429371,1402892,26479,9167,17312,1833,15479,684020,...,564.702724,1598.081783,25.820801,364.165256,4146.399781,181785.438970,10445.401951,31892.368304,32772,0.060540
3,0,0,1468671,1441306,27365,9285,18080,2114,15966,702921,...,530.355235,1532.790983,13.344249,329.520333,3993.609784,183071.107372,9158.312418,31892.368304,33986,0.060615
4,0,0,1484959,1457044,27915,9414,18501,2231,16270,712057,...,494.471686,1462.895553,5.048185,299.571229,3708.560637,183815.549825,8414.958660,31892.368304,37921,0.065916
5,0,0,1503240,1475568,27672,9358,18314,2191,16123,724260,...,249.285710,1353.676921,3.095452,207.552363,3447.896819,184880.914670,7354.584450,31892.368304,40796,0.068602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1999,0,3953,3953,0,0,0,0,0,1665,...,2.282785,64.510573,3.665569,0.000000,0.000000,369.734594,5.536110,0.000000,46,0.036249
612,1999,0,3922,3922,0,0,0,0,0,1660,...,2.049840,64.306363,3.376429,0.000000,0.000000,369.734594,5.536110,0.000000,48,0.037588
613,1999,0,3906,3906,0,0,0,0,0,1659,...,1.583949,63.747221,2.764354,0.000000,0.000000,369.734594,5.536110,0.000000,57,0.044049
614,1999,0,3828,3828,0,0,0,0,0,1641,...,1.506301,62.987953,2.585990,0.000000,0.000000,369.734594,5.536110,0.000000,58,0.044684


------------------------------------------------------------------
luz (1470, 127)
******************************************************************


,scenario,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,hs,...,redev_ag_roads,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,vac,unusable,vacancy,vacancy_rate
1,0,7162,7121,41,0,41,0,41,4715,3878,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,14.190585,59.487941,211,0.054409
2,0,7155,7114,41,0,41,1,40,4702,3883,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,14.190585,59.487941,192,0.049446
3,0,7155,7114,41,0,41,1,40,4701,3898,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,14.190585,59.487941,179,0.045921
4,0,7273,7231,42,0,42,2,40,4774,3918,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,14.190585,59.487941,114,0.029096
5,0,7255,7213,42,0,42,2,40,4762,3935,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,14.190585,59.487941,90,0.022872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,0,10565,10497,68,0,68,1,67,4958,4024,...,0.0,3.840816,93.776818,93.047038,22.499032,8.162603,17.742480,0.000000,232,0.057654
1711,0,9694,9625,69,0,69,1,68,4557,4226,...,0.0,3.840816,92.615707,90.385080,6.587490,8.162603,14.949968,0.000000,713,0.168717
1712,0,9676,9606,70,0,70,2,68,4558,4528,...,0.0,0.369939,88.284980,80.604189,0.000000,8.162603,12.442648,0.000000,989,0.218419
1713,0,9849,9780,69,0,69,2,67,4667,4565,...,0.0,0.369939,84.820773,76.973511,0.000000,8.162603,11.872241,0.000000,893,0.195619


------------------------------------------------------------------
taz (29610, 127)
******************************************************************


,scenario,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,hs,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
1,0,4570,2816,1754,1754,0,0,0,1210,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,66,0.066465
2,0,4549,2788,1761,1761,0,0,0,1200,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,64,0.064451
3,0,4535,2760,1775,1775,0,0,0,1192,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,63,0.063444
4,0,4528,2741,1787,1787,0,0,0,1188,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,60,0.060423
5,0,4463,2682,1781,1781,0,0,0,1171,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,61,0.061430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34540,0,5,5,0,0,0,0,0,5,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,6,0.750000
34541,0,0,0,0,0,0,0,0,0,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,8,1.000000
34542,0,0,0,0,0,0,0,0,0,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,8,1.000000
34543,0,0,0,0,0,0,0,0,0,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,8,1.000000
